In [1]:
import yfinance as yf
import pandas as pd
import time

# ✅ Full US Stock Universe (Sector-Wise 50 Stocks)
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# ✅ Data Storage List
stock_data = []

# ✅ Fetch Data for Each Stock
for sector, stocks in us_stock_universe.items():
    for stock in stocks:
        try:
            ticker = yf.Ticker(stock)

            # ✅ Fetch Last Closing Price
            hist = ticker.history(period="1d")
            last_price = hist["Close"].iloc[-1] if not hist.empty else "N/A"

            # ✅ Fetch Current P/E Ratio
            current_pe = ticker.info.get("trailingPE", "N/A")

            # ✅ Fetch Historical Avg P/E Ratio (Past 3 Years)
            eps_values = ticker.financials.loc["Diluted EPS"].dropna() if "Diluted EPS" in ticker.financials.index else pd.Series()
            avg_pe = (last_price / eps_values).mean() if not eps_values.empty else "N/A"

            # ✅ Fetch Direct Net Worth (Shareholder's Equity)
            net_worth = ticker.balance_sheet.loc["Total Stockholder Equity"].iloc[0] if "Total Stockholder Equity" in ticker.balance_sheet.index else "N/A"

            # ✅ If Direct Net Worth is N/A, Approximate It
            if net_worth == "N/A":
                market_cap = ticker.info.get("marketCap", "N/A")
                price_to_book = ticker.info.get("priceToBook", "N/A")

                if market_cap != "N/A" and price_to_book != "N/A" and price_to_book > 0:
                    net_worth = market_cap / price_to_book  # **Estimate Net Worth**
                else:
                    net_worth = "N/A"

            # ✅ Convert Net Worth to Billion $
            net_worth_billion = round(net_worth / 1e9, 2) if net_worth != "N/A" else "N/A"

            # ✅ Store Data
            stock_data.append({
                "Sector": sector,
                "Stock": stock,
                "Last Price ($)": round(last_price, 2) if last_price != "N/A" else "N/A",
                "Current P/E Ratio": round(current_pe, 2) if current_pe != "N/A" else "N/A",
                "Historical Avg P/E Ratio": round(avg_pe, 2) if avg_pe != "N/A" else "N/A",
                "Net Worth ($B)": net_worth_billion
            })

            print(f"✅ {stock} - Price: {last_price}, P/E: {current_pe}, Avg P/E: {avg_pe}, Net Worth: {net_worth_billion}")

        except Exception as e:
            print(f"❌ Error fetching {stock}: {e}")

        time.sleep(2)  # ✅ API Rate Limit Avoid Karne ke liye

# ✅ Convert to Pandas DataFrame
df = pd.DataFrame(stock_data)

# ✅ Print as Table
print(df)

# ✅ Save to CSV File (Optional)
df.to_csv("stock_prices.csv", index=False)


✅ AAPL - Price: 223.85000610351562, P/E: 35.588238, Avg P/E: 37.468297291703564, Net Worth: 66.67
✅ MSFT - Price: 390.5799865722656, P/E: 31.447664, Avg P/E: 40.61075850661125, Net Worth: 302.65
✅ GOOGL - Price: 162.24000549316406, P/E: 20.154037, Avg P/E: 36.95842959454272, Net Worth: 326.18
✅ NVDA - Price: 111.43000030517578, P/E: 37.90136, Avg P/E: 269.1097946545839, Net Worth: 79.08
✅ META - Price: 602.5800170898438, P/E: 25.25482, Avg P/E: 44.92184625384162, Net Worth: 182.61
✅ JPM - Price: 248.1199951171875, P/E: 12.563038, Avg P/E: 16.13179184117395, Net Worth: 324.53
✅ BAC - Price: 42.560001373291016, P/E: 13.258567, Avg P/E: 13.08500297566473, Net Worth: 272.11
✅ WFC - Price: 72.27999877929688, P/E: 13.459963, Avg P/E: 54.8475362137817, Net Worth: 159.35
✅ C - Price: 71.8499984741211, P/E: 12.07563, Avg P/E: 12.4720825914699, Net Worth: 191.27
✅ GS - Price: 558.9199829101562, P/E: 13.793681, Avg P/E: 16.555217905163282, Net Worth: 104.17
✅ PG - Price: 168.7100067138672, P/E: 2

In [2]:
# S&P 500 Sector ETFs
sector_name = {
    "Technology": "XLK",
    "Financials": "XLF",
    "Consumer Goods (Staples)": "XLP",
    "Healthcare": "XLV",
    "Energy": "XLE",
    "Industrials (Aerospace)": "XLI",
    "Consumer Discretionary": "XLY",
    "Telecom & Media": "XLC",
    "Utilities": "XLU",
    "Real Estate": "XLRE",
    "Materials": "XLB"
}

# Fetch data for all sector ETFs
def get_sp500_sector_etf_data():
    data = {}
    for sector, ticker in sector_name.items():
        stock = yf.Ticker(ticker)
        hist = stock.history(period='1d')  # Fetch latest daily data
        if not hist.empty:
            data[sector] = {
                "Last Price": hist['Close'].iloc[-1],
                "Open Price": hist['Open'].iloc[-1],
                "Highest": hist['High'].iloc[-1],
                "Lowest": hist['Low'].iloc[-1]
            }
        else:
            data[sector] = "Data not available"
    
    return pd.DataFrame(data).T

# Get sector ETF data and display
sector_etf_data = get_sp500_sector_etf_data()
print(sector_etf_data)
sector_etf_data.to_csv("sector_data.csv", index=False)



                          Last Price  Open Price     Highest      Lowest
Technology                211.509995  212.020004  213.220001  210.389999
Financials                 50.180000   50.189999   50.450001   49.790001
Consumer Goods (Staples)   80.809998   80.129997   80.970001   80.070000
Healthcare                144.970001  144.520004  145.419998  144.460007
Energy                     92.870003   93.500000   93.849998   92.500000
Industrials (Aerospace)   132.889999  133.559998  134.009995  132.270004
Consumer Discretionary    203.360001  202.589996  206.429993  202.080002
Telecom & Media            98.650002   98.760002   99.570000   98.550003
Utilities                  77.410004   77.360001   78.050003   77.180000
Real Estate                41.430000   41.590000   41.919998   41.325001
Materials                  86.639999   86.430000   86.940002   85.830002
